# Idea
    at first we want to extract a features and based on that  get the info

First, which model should we choose for the Image pretrining?
if you ask LLM's VGG is no brainer, thats because they have feed too many github repos which uses VGG to solve this problem, but in reality we can far better and newer models, as for me , i am choosing efficentNets, smaller lower FLOP's, lower resources better results,![image.png](attachment:image.png),

i am using collab, assume you have the zip file on collab


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Change to the directory where your zip file is located (e.g., My Drive)
%cd /content/drive/My Drive/

If you already have the unzipped data on your google drive, you can avoid running the next cell. Just make sure that the directory paths are correct in the config.

In [ ]:
# Unzip the file
!unzip caption_data.zip

In [ ]:
import pickle
import re
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.cuda.amp import autocast, GradScaler
import torchvision.transforms as T
from torchvision import models
from PIL import Image
import pandas as pd
import numpy as np
from collections import Counter
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42) # this should be automatically called by torch.manual_seed(), but just to make sure
print(f"Using device: {device}")

In [ ]:
# could be modified
class Config:
    # Paths
    IMAGE_DIR = "caption_data/Images"
    CAPTIONS_FILE = "caption_data/captions.txt"
    FEATURES_PATH = "caption_data/resnet_features.pt"
    VOCAB_PATH = "vocab.pkl"

    # Model
    EMBED_SIZE = 256
    HIDDEN_SIZE = 512
    NUM_LAYERS = 1
    DROPOUT = 0.5

    NUM_WORKERS = 2 if torch.cuda.is_available else 0

    # Training
    BATCH_SIZE = 64
    EPOCHS = 20
    LEARNING_RATE = 3e-4
    WEIGHT_DECAY = 1e-5
    GRAD_CLIP = 5.0

    # Data
    FREQ_THRESHOLD = 5  # could be modified
    MAX_SEQ_LEN = 50 # I don't think we need to use this

    # Splits
    TRAIN_SPLIT = 0.8
    VAL_SPLIT = 0.1

cfg = Config()

In [ ]:
train_transform = T.Compose([
    T.Resize((260, 260)),
    # T.RandomCrop((224, 224)), we can do that, but what if it crops the small neededed object
    T.RandomHorizontalFlip(p=0.5),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    # T.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)), too brutal
    T.RandomGrayscale(p=0.02), # we can delete it because the color is important for us, we do not grey sky output
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # efficent Net is using Imagenet data for the training so using these values here wont be a data leakage because these are the values of the Imagenet, even though our data could be different this values work well
])

# no shenanigans for this one
val_transform = T.Compose([
    T.Resize((260, 260)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:
class EfficientNetEncoder(nn.Module):
    """
    EfficientNet-B2 feature extractor - COMPLETELY FROZEN
    Only the projection layer is trainable
    """
    def __init__(self, embed_size):
        super().__init__()

        # Load pretrained EfficientNet-B2
        effnet = models.efficientnet_b2(
            weights=models.EfficientNet_B2_Weights.IMAGENET1K_V1
        )

        # EfficientNet structure:
        # features -> avgpool -> classifier
        self.backbone = effnet.features
        self.avgpool = effnet.avgpool

        # Feature dimension for B2
        backbone_out = 1408

        # Freeze backbone
        for param in self.backbone.parameters():
            param.requires_grad = False

        # Projection head (trainable)
        self.projection = nn.Sequential(
            nn.Flatten(),
            nn.Linear(backbone_out, embed_size),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

    def forward(self, images):
        with torch.no_grad():
            x = self.backbone(images)
            x = self.avgpool(x)

        features = self.projection(x)
        return features


## Vocabulary, Dataset and DataLoader
The next step is to write the Vocabulary logic that assigns unique integers to words and a dataloader that loads an image and its captions, preprocesses them and hands them to the model.

In [ ]:
class Vocabulary:
  def __init__(self, freq_threshold):
    self.itos = {0: "<pad>", 1: "<start>", 2: "<end>", 3: "<unk>"} # integers to tokens for specific token values
    # make sure that padding index is 0. it's important later on
    self.stoi = {"<pad>": 0, "<start>": 1, "<end>": 2, "<unk>": 3} # token to integers
    self.freq_threshold = freq_threshold # only recognize words with abs frequency >= freq_threshold

  def __len__(self):
    return len(self.itos)

  @staticmethod
  def tokenizer_eng(text):
    """
    Simple tokenizer: lowercase, remove punctuation, split by space.
    """
    # This can be replaced with a library tokenizer
    # I am writing a simple custom one for now, as I think sub-word tokenization is not needed for our use-case
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", "", text) # NOTE: we might need to write special logic for handling numeric tokens
    text = re.sub(r"\s+", " ", text).strip()
    return text.split(" ")

  def build_vocab(self, sentence_list):
    """
    Build a vocabulary of frequent words in sentence_list
    """
    frequencies = {}
    idx = 4 # we already have 4 words in our vocab (special tokens)

    # count the frequencies of all the words
    # NOTE: we might want to cap the vocabulary size. I am not doing it for now as I think it won't be needed for such a small dataset
    for sentence in sentence_list:
      for word in self.tokenizer_eng(sentence):
        if word not in frequencies: frequencies[word] = 1
        else: frequencies[word] += 1

    # assign an unique integer to words with abs frequency >= freq threshold
    # for determinism, sort by frequency (descending) and then alphabetically (ascending)
    sorted_words = sorted(frequencies.items(), key=lambda item: (-item[1], item[0]))
    for word, count in sorted_words:
      if count >= self.freq_threshold:
        self.stoi[word] = idx
        self.itos[idx] = word
        idx += 1

  def numericalize(self, text):
    """
    Convert a text string into a list of integers
    """
    # convert a text string into a list of integers
    tokenized_text = self.tokenizer_eng(text)
    return [self.stoi[token] if token in self.stoi else self.stoi["<unk>"] for token in tokenized_text]

  def decode(self, ids, skip_special_tokens=True):
    """
    Reconstruct a sentence from integer vector
    """
    if skip_special_tokens:
        tokens = [self.itos.get(i, "<unk>") for i in ids if i not in [0, 1, 2]]  # skip <pad>, <start>, <end>
    else:
        tokens = [self.itos.get(i, "<unk>") for i in ids]
    return " ".join(tokens)

  def get_stats(self):
    return {
        "vocab_size": len(self),
        "num_special_tokens": 4,
        "num_words": len(self) - 4
    }

  def save_vocabulary(self, filename):
    """
    Saves vocabulary mappings to a file.
    """
    # use pickle instead of json so that integer keys are saved as integers
    with open(filename, "wb") as f:
      pickle.dump({
          "stoi": self.stoi,
          "itos": self.itos,
          "freq_threshold": self.freq_threshold
      }, f)
    print(f"Vocabulary saved to {filename}")

  @staticmethod
  def load_vocabulary(filename):
    """
    Loads a vocabulary from a file and returns a Vocabulary object
    """
    with open(filename, "rb") as f:
      data = pickle.load(f)

    vocab = Vocabulary(freq_threshold=data["freq_threshold"])
    vocab.stoi = data["stoi"]
    vocab.itos = data["itos"]

    print(f"Vocabulary loaded from {filename}")
    return vocab

In [ ]:
class CaptionDataset(Dataset):
  def __init__(self, root_dir, images_list, captions_list, transform=None, freq_threshold=5, vocab=None):
    self.root_dir = root_dir
    self.transform = transform

    if not os.path.exists(root_dir):
      raise FileNotFoundError(f"Image directory not found: {root_dir}")

    self.imgs = images_list
    self.captions = captions_list

    # if a vocabulary is not passed in, build it
    if vocab is not None: self.vocab = vocab
    else:
      self.vocab = Vocabulary(freq_threshold)
      self.vocab.build_vocab(self.captions)

  def __len__(self):
    return len(self.captions)

  def __getitem__(self, index):
    # get image caption + id (which in this case is the file name)
    caption = self.captions[index]
    img_id = self.imgs[index]

    # load image
    img_path = os.path.join(self.root_dir, img_id)
    if not os.path.exists(img_path):
      raise FileNotFoundError(f"Image not found: {img_path}")

    img = Image.open(img_path).convert("RGB")


    # apply transformations
    if self.transform is not None: img = self.transform(img)

    # convert caption into a list of integers
    numericalized_caption = [self.vocab.stoi["<start>"]] + self.vocab.numericalize(caption) + [self.vocab.stoi["<end>"]]

    return img, torch.tensor(numericalized_caption)

We cannot use the default pytorch data loader because the caption sentences are of different lengths and they cannot be stacked on top of each other. We need a custom function that will pad the batch to the length of the longest sentence in the batch.

In [ ]:
class MyCollate:
  def __init__(self, pad_idx):
    self.pad_idx = pad_idx

  def __call__(self, batch):
    batch.sort(key=lambda x: len(x[1]), reverse=True)
    # a batch is a list of tuples (image, caption)
    imgs = [item[0] for item in batch]
    captions = [item[1] for item in batch]

    # stack images (they are all already resized to the same size through transform)
    imgs = torch.stack(imgs, dim=0)

    # pad captions. pad_sequence handles both padding and stacking
    targets = pad_sequence(captions, batch_first=True, padding_value=self.pad_idx)

    lengths = torch.tensor([len(c) for c in captions])

    return imgs, targets, lengths

In [ ]:
# stratified train/val split to make sure that images are split between train/val to avoid data leakage
df = pd.read_csv(cfg.CAPTIONS_FILE)
unique_images = df["image"].unique()
np.random.shuffle(unique_images)
train_img_count = int(cfg.TRAIN_SPLIT * len(unique_images))
train_images = set(unique_images[:train_img_count])
val_images = set(unique_images[train_img_count:])

all_images = df["image"].tolist()
train_indices = [i for i, img in enumerate(all_images) if img in train_images]
val_indices = [i for i, img in enumerate(all_images) if img in val_images]

print(f"Train images: {len(train_images)}, Val images: {len(val_images)}")
print(f"Train captions: {len(train_indices)}, Val captions: {len(val_indices)}")

# build vocab only on training captions to avoid data leakage
all_captions = df["caption"].tolist()
train_captions = [all_captions[i] for i in train_indices]
vocab = Vocabulary(cfg.FREQ_THRESHOLD)
vocab.build_vocab(train_captions)
# vocab.save_vocabulary(cfg.VOCAB_PATH) # uncomment to save the vocabulary

print(f"Vocabulary size: {len(vocab)}")

train_dataset = CaptionDataset(
    root_dir=cfg.IMAGE_DIR,
    images_list=all_images,
    captions_list=all_captions,
    transform=train_transform,
    freq_threshold=cfg.FREQ_THRESHOLD,
    vocab=vocab
)

val_dataset = CaptionDataset(
    root_dir=cfg.IMAGE_DIR,
    images_list=all_images,
    captions_list=all_captions,
    transform=val_transform,
    freq_threshold=cfg.FREQ_THRESHOLD,
    vocab=vocab
)

train_subset = Subset(train_dataset, train_indices)
val_subset = Subset(val_dataset, val_indices)

pad_idx = vocab.stoi["<pad>"]

train_loader = DataLoader(
    train_subset,
    batch_size=cfg.BATCH_SIZE,
    shuffle=True,
    num_workers=cfg.NUM_WORKERS,
    collate_fn=MyCollate(pad_idx=pad_idx),
    pin_memory=True,
    persistent_workers=True if cfg.NUM_WORKERS > 0 else False
)

val_loader = DataLoader(
    val_subset,
    batch_size=cfg.BATCH_SIZE,
    shuffle=False,
    num_workers=cfg.NUM_WORKERS,
    collate_fn=MyCollate(pad_idx=pad_idx),
    pin_memory=True,
    persistent_workers=True if cfg.NUM_WORKERS > 0 else False
)

print(f"Train samples: {len(train_subset)}, Val samples: {len(val_subset)}")

## Decoder
Now, it's time to implement the decoder.

In [ ]:
class DecoderRNN(nn.Module):
  def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1, dropout=0.5):
    super(DecoderRNN, self).__init__()

    # embedding layer
    self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0) # num of embeddings = vocab_size, length of embed vector = embed_size

    # lstm layer
    self.lstm = nn.LSTM(
        input_size=embed_size,
        hidden_size=hidden_size,
        num_layers=num_layers,
        batch_first=True,
        dropout=dropout if num_layers > 1 else 0.0
    )

    # linear layer to convert lstm output to words score
    self.linear = nn.Linear(hidden_size, vocab_size)

    self.dropout = nn.Dropout(dropout)

  def forward(self, features, captions, lengths):
    """
    features: the image vectors (batch_size, embed_size)
    captions: word indices (batch_size, seq_length)
    lengths: the lengths of the sequences (batch_size)
    """
    # embed the captions, ignore the <end> token
    embeds = self.dropout(self.embedding(captions[:, :-1])) #dimensions: (batch_size, seq_length - 1, embed_size)

    # turn image features into (batch_size, 1, embed_size) size
    features = features.unsqueeze(1)

    #treat the features vector as the first word in sequence
    inputs = torch.cat((features, embeds), dim=1) #dimensions: (batch_size, seq_length, embed_size)

    packed = pack_padded_sequence(inputs, lengths, batch_first=True, enforce_sorted=False)

    hiddens, _ = self.lstm(packed)
    hiddens, _ = pad_packed_sequence(hiddens, batch_first=True) # hiddens dimensions: (batch_size, sequence_length, hidden_size)
    outputs = self.linear(hiddens) #dimensions: (batch_size, sequence_length, vocab_size)
    return outputs


In [ ]:
class CNNtoRNN(nn.Module):
  def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1):
    super(CNNtoRNN, self).__init__()
    self.encoder = EfficientNetEncoder(embed_size)
    self.decoder = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers)

  def forward(self, images, captions, lengths):
    features = self.encoder(images)
    outputs = self.decoder(features, captions, lengths)
    return outputs

  def caption_image(self, image, vocabulary, max_length=50):
    """
    image: input image tensor (batch, channel, height, width)
    vocabulary: the Vocabulary object
    max_length: the maximum length of the caption output
    """
    self.eval()
    result_caption = []
    with torch.no_grad():
      # get image features
      x = self.encoder(image) # dim: (1, embed_size)
      x = x.unsqueeze(1) # dim: (1, 1, embed_size)

      # Initial LSTM states
      states = None

      # run the image through the LSTM
      hiddens, states = self.decoder.lstm(x, states) # hiddens dim: (1, 1, hidden_size)

      # start the loop with the <start> token
      start_token_idx = vocabulary.stoi["<start>"]

      next_word_input = torch.tensor([[start_token_idx]]).to(image.device)

      for _ in range(max_length):
        # embed the current word input
        embeds = self.decoder.embedding(next_word_input) # dim: (1, 1, embed_size)

        # run lstm, feed the states from the previous step back in
        hiddens, states = self.decoder.lstm(embeds, states) # dim: (1, 1, hidden_size)

        # calculate the output probabilities
        output = self.decoder.linear(hiddens[:, -1, :]) # dim: (1, vocab_size)

        # pick the word with the highest probability NOTE: this can be changed
        predicted = output.argmax(dim=1) # dim: (1)
        item = predicted.item()

        if item == vocabulary.stoi["<end>"]: break # if predicted <end> token, stop

        word = vocabulary.itos[item]
        if word not in ["<start>", "<pad>"]:
          result_caption.append(word)

        next_word_input = predicted.unsqueeze(1) # dim: (1, 1)

      return result_caption
